In [1]:
import os
import numpy as np

from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
gpus = tf.config.list_physical_devices('GPU')
try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [6]:
img_augmentation = Sequential(
    [
     preprocessing.RandomRotation(factor=0.15),
     preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
     preprocessing.RandomFlip(),
     preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [7]:
inputs = layers.Input(shape=(32, 32, 3))

x = layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(inputs)
x = img_augmentation(x)

basic_model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(basic_model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)

# Compile
model = tf.keras.Model(inputs, outputs, name="EfficientNet")

# initiate Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

checkpoint_path = './models/checkpoint.ckpt'

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True)
]

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
img_augmentation (Sequential)   (None, 224, 224, 3)  0           lambda[0][0]                     
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           img_augmentation[0][0]           
_______________________________________________________________________________________

In [ ]:
# Setup some hyperparameters
batch_size = 16
epochs = 50

# Fit the data into model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True,
          callbacks=callbacks)

Epoch 1/50
 361/3125 [==>...........................] - ETA: 11:24 - loss: 1.7725 - accuracy: 0.4893

In [ ]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

In [ ]:
y_pred[0] 

In [ ]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

## Build a Class for Prediction

In [ ]:
class MyModel:
    def __init__(self, model):
        self.model = model
    
    def predict(self, x):
        return np.argmax(self.model.predict(x), axis=1)

In [ ]:
best_model = keras.models.load_model(checkpoint_path)

your_model = MyModel(best_model)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
y_pred = your_model.predict(x_test)

In [ ]:
assert y_pred.shape == (10000,)

In [ ]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))